In [209]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
pd.set_option("display.max_columns", 100)
import os
import geojson
from geojson import Feature, FeatureCollection
import requests
from bs4 import BeautifulSoup
import json

### Get starting and ending indices of each sheet

In [130]:
f = open("../patientdb", "r")
res = f.read()
soup1 = BeautifulSoup(res,'lxml')
all_tr = soup1.findAll("tr")


In [158]:
start_idx_rawData = -1
end_idx_rawData = -1

start_idx_deathRecovered = -1
end_idx_deathRecovered = -1

start_idx_stateWise = -1
end_idx_stateWise = -1

start_idx_stateWiseTesting = -1
end_idx_stateWiseTesting = -1

start_idx_districtWise = -1
end_idx_districtWise = -1

start_idx_casesTimeSeries = -1
end_idx_casesTimeSeries = -1

start_idx_stateWiseDaily = -1
end_idx_stateWiseDaily = -1

#start_idx_mhdata = -1
#end_idx_mhdata = -1

start_idx_icmrTest = -1
end_idx_icmrTest = -1

start_idx_travelHistory = -1
end_idx_travelHistory = -1

for i in range(len(all_tr)):
    tr_td = all_tr[i].findAll("td")

    if (len(tr_td)>2):
        first_td_text = tr_td[0].text
        second_td_text = tr_td[1].text
        third_td_text = tr_td[2].text

        if ((first_td_text=="Patient Number") and (second_td_text=="State Patient Number")):
            print(first_td_text + " " + str(i))
            start_idx_rawData = i+2
        if ((first_td_text=="Sl_No") and (second_td_text=="Date")):
            print(first_td_text + " " + str(i))
            start_idx_deathRecovered = i+2
            end_idx_rawData = i-2
        if ((first_td_text=="State")): # and (second_td_text=="Confirmed")):
            print(first_td_text + " " + str(i))
            start_idx_stateWise = i+3  # i+1 will include total row (i+2 is empty)
            end_idx_deathRecovered = i-2
        if ((first_td_text=="Updated On") and (second_td_text=="State")):
            print(first_td_text + " " + str(i))
            start_idx_stateWiseTesting = i+2
            end_idx_stateWise = i-2
        if ((first_td_text=="District") and (second_td_text=="State")):
            print(first_td_text + " " + str(i))
            start_idx_districtWise = i+2
            end_idx_stateWiseTesting = i-2
        if ((first_td_text=="Date") and (second_td_text=="Daily Confirmed")):
            print(first_td_text + " " + str(i))
            start_idx_casesTimeSeries = i+1    # i+1 will include 30 jan, i+2 is empty
            end_idx_districtWise = i-2
        if ((first_td_text=="Date") and (second_td_text=="Status") and (third_td_text!="TT")):
            print(first_td_text + " " + str(i))
            start_idx_stateWiseDaily = i+2   
            end_idx_casesTimeSeries = i-2
        if ((first_td_text=="District") and (second_td_text=="") and (third_td_text=="6/4/2020")):
            print(first_td_text + " " + str(i))
            #start_idx_mhdata = i+2   
            end_idx_stateWiseDaily = i-2
        if ((first_td_text=="Update Time Stamp") and (second_td_text=="Total Samples Tested")):
            print(first_td_text + " " + str(i))
            start_idx_icmrTest = i+2

            row_idx = i+2
            row_data = all_tr[row_idx]
            first_col_text = row_data.find_all("td")[0].text
            end_idx_icmrTest = row_idx
            while (first_col_text.endswith("pm") or first_col_text.endswith("am")):
                row_idx+=1
                end_idx_icmrTest = row_idx-1
                row_data = all_tr[row_idx]
                if len(row_data.find_all("td"))>0:
                    first_col_text = row_data.find_all("td")[0].text
                else:
                    first_col_text = ""
            
        if ((second_td_text=="PID")):
            print(second_td_text + " " + str(i))
            start_idx_travelHistory = i+1  

        if ((first_td_text=="N") and (second_td_text=="District")):
            print(first_td_text + " " + str(i))
            end_idx_travelHistory = i-2

Patient Number 1
Sl_No 8104
State 9526
Updated On 9567
District 10522
Date 11252
Date 11326
District 11410
Update Time Stamp 11561
PID 11700
N 11941


### Create files from each sheet

In [116]:
########################### Raw Data ##############################
rows_rawData = all_tr[start_idx_rawData: end_idx_rawData]
df = pd.DataFrame()
for row_idx in range(len(rows_rawData)):
    row = rows_rawData[row_idx].findAll("td")
    df.at[row_idx, "patient_number"] = row[0].text
    df.at[row_idx, "date"] = row[2].text
    df.at[row_idx, "age"] = row[3].text
    df.at[row_idx, "gender"] = row[4].text            
    df.at[row_idx, "city"] = row[5].text
    df.at[row_idx, "district"] = row[6].text
    df.at[row_idx, "state"] = row[7].text
    df.at[row_idx, "state_code"] = row[8].text
    df.at[row_idx, "state_patient_number"] = row[1].text
    df.at[row_idx, "current_status"] = row[9].text
    df.at[row_idx, "contracted_from"] = row[11].text
    df.at[row_idx, "nationality"] = row[12].text
    df.at[row_idx, "transmission_type"] = row[13].text
    df.at[row_idx, "status_updated_on"] = row[14].text
    df.at[row_idx, "notes"] = row[10].text

df = df[df["state"]!=""]
df.to_csv("../data/extracted_data/raw_data.csv", index=False)

In [118]:
########################### Death Recovered ##############################
rows_data = all_tr[start_idx_deathRecovered: end_idx_deathRecovered]
df = pd.DataFrame()
for row_idx in range(len(rows_data)):
    row = rows_data[row_idx].findAll("td")
    df.at[row_idx, "sl_no"] = row[0].text
    df.at[row_idx, "date"] = row[1].text
    df.at[row_idx, "age"] = row[2].text
    df.at[row_idx, "gender"] = row[3].text            
    df.at[row_idx, "status"] = row[4].text
    df.at[row_idx, "city"] = row[5].text
    df.at[row_idx, "district"] = row[6].text
    df.at[row_idx, "state"] = row[7].text
    df.at[row_idx, "state_code"] = row[8].text
    df.at[row_idx, "nationality"] = row[10].text
    df.at[row_idx, "patient_number"] = row[14].text
    df.at[row_idx, "notes"] = row[9].text

df = df[df["state"]!=""]
df.to_csv("../data/extracted_data/death_recovered.csv", index=False)

In [122]:
########################### State wise ##############################
rows_data = all_tr[start_idx_stateWise: end_idx_stateWise]
df = pd.DataFrame()
for row_idx in range(len(rows_data)):
    row = rows_data[row_idx].findAll("td")
    df.at[row_idx, "state"] = row[0].text
    df.at[row_idx, "confirmed"] = row[2].text
    df.at[row_idx, "recovered"] = row[3].text            
    df.at[row_idx, "deaths"] = row[4].text
    df.at[row_idx, "active"] = row[5].text
    df.at[row_idx, "last_updated"] = row[6].text
    df.at[row_idx, "state_code"] = row[7].text

df = df[df["state"]!=""]
df.to_csv("../data/extracted_data/statewise.csv", index=False)

In [ ]:
########################### State wise ##############################
rows_data = all_tr[start_idx_stateWise: end_idx_stateWise]
df = pd.DataFrame()
for row_idx in range(len(rows_data)):
    row = rows_data[row_idx].findAll("td")
    df.at[row_idx, "state"] = row[0].text
    df.at[row_idx, "confirmed"] = row[2].text
    df.at[row_idx, "recovered"] = row[3].text            
    df.at[row_idx, "deaths"] = row[4].text
    df.at[row_idx, "active"] = row[5].text
    df.at[row_idx, "last_updated"] = row[6].text
    df.at[row_idx, "state_code"] = row[7].text

df = df[df["state"]!=""]
df.to_csv("../data/extracted_data/statewise.csv", index=False)

In [135]:
######################## State wise testing ########################
rows_data = all_tr[start_idx_stateWiseTesting: end_idx_stateWiseTesting]
df = pd.DataFrame()
for row_idx in range(len(rows_data)):
    row = rows_data[row_idx].findAll("td")
    df.at[row_idx, "last_updated"] = row[0].text
    df.at[row_idx, "state"] = row[1].text
    df.at[row_idx, "total_tested"] = row[2].text
    df.at[row_idx, "positive"] = row[3].text            
    df.at[row_idx, "negative"] = row[4].text
    df.at[row_idx, "unconfirmed"] = row[5].text
    df.at[row_idx, "num_in_quarantine"] = row[6].text
    df.at[row_idx, "num_released_from_quarantine"] = row[7].text
    df.at[row_idx, "num_isolation_beds"] = row[8].text
    df.at[row_idx, "num_icu_beds"] = row[9].text
    df.at[row_idx, "num_ventilators"] = row[10].text

df = df[df["state"]!=""]
df.to_csv("../data/extracted_data/statewise_testing.csv", index=False)

In [138]:
######################## District wise ########################
rows_data = all_tr[start_idx_districtWise: end_idx_districtWise]
df = pd.DataFrame()
for row_idx in range(len(rows_data)):
    row = rows_data[row_idx].findAll("td")
    df.at[row_idx, "district"] = row[0].text
    df.at[row_idx, "state"] = row[1].text
    df.at[row_idx, "confirmed"] = row[3].text
    df.at[row_idx, "recovered"] = row[4].text            
    df.at[row_idx, "deaths"] = row[5].text
    df.at[row_idx, "active"] = row[6].text
df = df[df["state"]!=""]
df.to_csv("../data/extracted_data/districtwise.csv", index=False)

In [187]:
######################## Case time series ########################
rows_data = all_tr[start_idx_casesTimeSeries: end_idx_casesTimeSeries+1]
df = pd.DataFrame()
for row_idx in range(len(rows_data)):
    row = rows_data[row_idx].findAll("td")
    df.at[row_idx, "date"] = row[0].text
    df.at[row_idx, "daily_confirmed"] = row[1].text
    df.at[row_idx, "total_confirmed"] = row[2].text
    df.at[row_idx, "daily_recovered"] = row[3].text
    df.at[row_idx, "total_recovered"] = row[4].text
    df.at[row_idx, "daily_deaths"] = row[5].text
    df.at[row_idx, "total_deaths"] = row[6].text
df.drop(index=1, inplace=True)
df.to_csv("../data/extracted_data/cases_time_series.csv", index=False)

In [162]:
######################## Statewise daily ########################
rows_data = all_tr[start_idx_stateWiseDaily: end_idx_stateWiseDaily+1]
df = pd.DataFrame()

col_names = []
header_row = all_tr[start_idx_stateWiseDaily-2].findAll("td")
for i in range(len(header_row)):
    col_names.append(header_row[i].text)
    
for row_idx in range(len(rows_data)):
    row = rows_data[row_idx].findAll("td")
    
    for col_idx in range(len(row)):
        if col_idx!=2:
            df.at[row_idx, col_names[col_idx]] = row[col_idx].text

df.to_csv("../data/extracted_data/statewise_daily.csv", index=False)

In [170]:
######################## ICMR testing ########################
rows_data = all_tr[start_idx_icmrTest: end_idx_icmrTest+1]
df = pd.DataFrame()
for row_idx in range(len(rows_data)):
    row = rows_data[row_idx].findAll("td")
    df.at[row_idx, "update_timestamp"] = row[0].text
    df.at[row_idx, "total_samples_tested"] = row[1].text
    df.at[row_idx, "total_individuals_tests"] = row[2].text
    df.at[row_idx, "total_positive_cases"] = row[3].text
    df.at[row_idx, "total_private_labs_tests"] = row[4].text
    df.at[row_idx, "sample_reported_today"] = row[5].text
    df.at[row_idx, "positive_cases_from_samples_reported"] = row[6].text
df.to_csv("../data/extracted_data/icmr_testing.csv", index=False)

In [176]:
######################## Travel history ########################
rows_data = all_tr[start_idx_travelHistory: end_idx_travelHistory+1]
df = pd.DataFrame()
for row_idx in range(len(rows_data)):
    row = rows_data[row_idx].findAll("td")
    df.at[row_idx, "sl_no"] = row[0].text
    df.at[row_idx, "pid"] = row[1].text
    df.at[row_idx, "time_from"] = row[2].text
    df.at[row_idx, "time_to"] = row[3].text
    df.at[row_idx, "address"] = row[4].text
    if row[5].text == "":
        df.at[row_idx, "lat"] = np.NAN
        df.at[row_idx, "long"] = np.NAN
    else:
        df.at[row_idx, "lat"] = row[5].text.split(",")[0]
        df.at[row_idx, "long"] = row[5].text.split(",")[1]
    df.at[row_idx, "location_accuracy"] = row[6].text
    df.at[row_idx, "type"] = row[7].text
    df.at[row_idx, "mode_of_travel"] = row[8].text
    df.at[row_idx, "place_name"] = row[9].text
    
df.to_csv("../data/extracted_data/travel_history.csv", index=False)

In [182]:
####################### State Code ######################
df = pd.read_csv("../data/copied_data/state_codes.csv", usecols=[1,2])
df.columns = ["state", "state_code"]
df.to_csv("../data/extracted_data/state_codes.csv", index=False)

In [186]:
####################### District list ######################
df = pd.read_csv("../data/copied_data/district_list.csv", usecols=[1,2,3,4])
df.columns = ["district", "state", "notes", "other_names"]
df.to_csv("../data/extracted_data/district_list.csv", index=False)

### Daily cases and deaths

In [202]:
df = pd.read_csv("../data/extracted_data/cases_time_series.csv", usecols=[0,1,2,5,6])
df["date"] = df["date"].str.strip()
for i in range(len(df)):
    df.at[i, "date"] = df.iloc[i]["date"] + " 2020"
df["date"] = pd.to_datetime(df["date"])

dfr = df.set_index("date").rolling(7).mean().reset_index()[["date", "daily_confirmed", "daily_deaths"]]
dfr.columns = ["date", "7day_avg_confirmed", "7day_avg_deaths"]
df = df.set_index("date").join(dfr.set_index("date")).reset_index()

df.to_csv("../code/data/overall_and_daily_cases_deaths.csv", index=False)
df.head(2)

,date,daily_confirmed,total_confirmed,daily_deaths,total_deaths,7day_avg_confirmed,7day_avg_deaths
0,2020-01-30,1,1,0,0,NaN,NaN
1,2020-01-31,0,1,0,0,NaN,NaN


### District level map

In [233]:
df = pd.read_csv("../data/extracted_data/districtwise.csv")
df.head(2)

,district,state,confirmed,recovered,deaths,active
0,Nicobars,Andaman and Nicobar Islands,0,0,0,0
1,North and Middle Andaman,Andaman and Nicobar Islands,1,0,0,1


In [286]:
df["confirmed"].sum()

5404

In [280]:
dfr = pd.read_csv("../data/extracted_data/raw_data.csv")

In [281]:
dfr.head(2)

,patient_number,date,age,gender,city,district,state,state_code,state_patient_number,current_status,contracted_from,nationality,transmission_type,status_updated_on,notes
0,1,30/01/2020,20,F,Thrissur,Thrissur,Kerala,KL,KL-TS-P1,Recovered,NaN,India,Imported,14/02/2020,Travelled from Wuhan
1,2,02/02/2020,NaN,NaN,Alappuzha,Alappuzha,Kerala,KL,KL-AL-P1,Recovered,NaN,India,Imported,14/02/2020,Travelled from Wuhan


In [282]:
dfr["current_status"].unique()

array(['Recovered', 'Hospitalized', 'Deceased', 'Migrated', nan],
      dtype=object)

In [283]:
len(dfr[dfr["current_status"]=="Hospitalized"])

6686

In [284]:
len(dfr)

6835

In [285]:
len(dfr[dfr["current_status"]=="Recovered"])

119

In [287]:
len(dfr[dfr["current_status"]=="Deceased"])

28

In [289]:
6686 + 28 + 119

6833

In [291]:
dfr.tail(2)

,patient_number,date,age,gender,city,district,state,state_code,state_patient_number,current_status,contracted_from,nationality,transmission_type,status_updated_on,notes
6833,6834,10/04/2020,35,M,NaN,Bengaluru Rural,Karnataka,KA,KA-P206,Hospitalized,KA-P169,NaN,NaN,NaN,Brother of KA-P169
6834,6835,10/04/2020,11,F,NaN,Bengaluru Rural,Karnataka,KA,KA-P207,Hospitalized,KA-P206,NaN,NaN,NaN,Daughter of KA-P206


In [ ]:
"transform": {
    "scale": [
      0.0006973698971092601,
0.0006462543480830687
    ],
    "translate": [
      76.20310065122152,
      8.107847033856163
    ]
  },

In [274]:
state_name_list = []

for fp in os.listdir("../data/state_geojson_raw/"):
    if not fp.startswith(".") and fp.endswith(".json"):
        state_name = fp.split(".")[0]
        state_name_list.append(state_name)
        print(state_name)

        with open("../data/state_geojson_raw/"+fp) as topodata:
            raw_data = json.load(topodata)

        data = raw_data
        district_list = data["objects"][state_name+"_district"]["geometries"]
        
        if state_name=="tamilnadu":
            data["transform"]["scale"] = [0.0006973698971092601, 0.0006462543480830687]
            data["transform"]["translate"] = [76.20310065122152, 8.107847033856163]
        
        for i in range(len(district_list)):
            district_name = data["objects"][state_name+"_district"]["geometries"][i]["properties"]["district"]
            df_district = df[df["district"]==district_name].copy()
            
            if len(df_district)==1:
                data["objects"][state_name+"_district"]["geometries"][i]["properties"]["confirmed"] = str(df_district["confirmed"].values[0])
                data["objects"][state_name+"_district"]["geometries"][i]["properties"]["recovered"] = str(df_district["recovered"].values[0])
                data["objects"][state_name+"_district"]["geometries"][i]["properties"]["deaths"] = str(df_district["deaths"].values[0])
                data["objects"][state_name+"_district"]["geometries"][i]["properties"]["active"] = str(df_district["active"].values[0])
                
                data["objects"][state_name+"_district"]["geometries"][i]["properties"]["confirmed_colorscale"] = str(df_district["confirmed"].values[0]/df["confirmed"].max())
                data["objects"][state_name+"_district"]["geometries"][i]["properties"]["recovered_colorscale"] = str(df_district["recovered"].values[0]/df["recovered"].max())
                data["objects"][state_name+"_district"]["geometries"][i]["properties"]["deaths_colorscale"] = str(df_district["deaths"].values[0]/df["deaths"].max())
                data["objects"][state_name+"_district"]["geometries"][i]["properties"]["active_colorscale"] = str(df_district["active"].values[0]/df["active"].max())

            elif len(df_district)==0:
                print(district_name + " data not found")
            elif len(df_district)>1:
                print(district_name + " more than one data found")
            else:
                print(district_name + " other error")

        with open('../data/state_geojson_data/'+state_name+'.json', 'w') as outfile:
            json.dump(data, outfile)

odisha
telangana
meghalaya
karnataka
haryana
bihar
Kaimur Bhabhua data not found
Aurangabad more than one data found
andhrapradesh
jammukashmir
westbengal
Uttar Dinajpur data not found
kerala
chhattisgarh
Raigarh more than one data found
Bilaspur more than one data found
Balrampur more than one data found
andamannicobarislands
jharkhand
Saraikela-kharsawan data not found
ladakh
uttarpradesh
Balrampur more than one data found
Hamirpur more than one data found
Pratapgarh more than one data found
mizoram
lakshadweep
nagaland
tamilnadu
Thoothukudi data not found
dadranagarhaveli
delhi
puducherry
madhyapradesh
arunachalpradesh
uttarakhand
manipur
tripura
gujarat
goa
assam
maharashtra
Aurangabad more than one data found
Raigarh more than one data found
punjab
sikkim
North  District data not found
rajasthan
Jalor data not found
Pratapgarh more than one data found
chandigarh
himachalpradesh
Hamirpur more than one data found
Bilaspur more than one data found


In [275]:
df["confirmed"].describe()

count    726.000000
mean       7.443526
std       37.620779
min        0.000000
25%        0.000000
50%        0.000000
75%        4.000000
max      876.000000
Name: confirmed, dtype: float64

In [277]:
df[df["confirmed"]>100]

,district,state,confirmed,recovered,deaths,active
156,Ahmadabad,Gujarat,153,3,6,144
303,Kasaragod,Kerala,161,5,0,156
338,Indore,Madhya Pradesh,173,0,15,158
384,Mumbai,Maharashtra,876,14,45,817
393,Pune,Maharashtra,212,16,14,182
522,Jaipur,Rajasthan,168,0,0,168
544,Chennai,Sikkim,166,0,4,162
577,Hyderabad,Tamil Nadu,104,0,4,100


In [279]:
df[df["district"]=="Chennai"]

,district,state,confirmed,recovered,deaths,active
544,Chennai,Sikkim,166,0,4,162


In [272]:
data["transform"]

{'scale': [0.0003775508127833683, 0.0003244601600360647],
 'translate': [75.594723, 30.377701000000002]}

In [256]:
df[df["state"]=="Himachal Pradesh"]

,district,state,confirmed,recovered,deaths,active
211,Bilaspur,Himachal Pradesh,1,0,0,1
212,Chamba,Himachal Pradesh,0,0,0,0
213,Hamirpur,Himachal Pradesh,0,0,0,0
214,Kangra,Himachal Pradesh,3,0,0,3
215,Kinnaur,Himachal Pradesh,0,0,0,0
216,Kullu,Himachal Pradesh,0,0,0,0
217,Lahul and Spiti,Himachal Pradesh,0,0,0,0
218,Mandi,Himachal Pradesh,0,0,0,0
219,Shimla,Himachal Pradesh,0,0,0,0
220,Sirmaur,Himachal Pradesh,1,0,0,1


In [255]:
df["state"].unique()

array(['Andaman and Nicobar Islands', 'Andhra Pradesh',
       'Arunachal Pradesh', 'Assam', 'Bihar', 'Chandigarh',
       'Chhattisgarh', 'Dadra and Nagar Haveli', 'Daman And Diu', 'Delhi',
       'Goa', 'Gujarat', 'Haryana', 'Himachal Pradesh',
       'Jammu and Kashmir', 'Jharkhand', 'Karnataka', 'Kerala', 'Ladakh',
       'Lakshadweep', 'Madhya Pradesh', 'Maharashtra', 'Manipur',
       'Meghalaya', 'Mizoram', 'Nagaland', 'Odisha', 'Puducherry',
       'Punjab', 'Rajasthan', 'Sikkim', 'Tamil Nadu', 'Telangana',
       'Tripura', 'Uttar Pradesh', 'Uttarakhand', 'West Bengal'],
      dtype=object)

In [270]:
a = 0.265
print(0.0026315845173934343 * a)
print(0.0024386956531436553 * a)

0.0006973698971092601
0.0006462543480830687


In [ ]:
"""
Scale: 0.265 of original values
Tamil nadu adjusted final values:
    "transform": {
    "scale": [
      0.0006973698971092601,
0.0006462543480830687
    ],
    "translate": [
      76.20310065122152,
      8.107847033856163
    ]
  },
"""